In [1]:
import torch
import torchvision
from torchvision import transforms
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import os

# Classification Model
From the scratch Classification Model for CIFAR-10 Classification.

In [4]:
class CNN_classifier(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.block1 = nn.Sequential(
            nn.Conv2d(in_channels, 32, kernel_size=5, padding=2, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, kernel_size=5, padding=2, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Dropout(0.10)
        )
        self.block2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, padding=2, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=5, padding=2, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Dropout(0.15)
        )
        self.block3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=5, padding=2, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=5, padding=2, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Dropout(0.20)
        )
        self.block4 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=5, padding=2, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=5, padding=2, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Dropout(0.30)
        )
        self.head = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(256, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.40),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        y = self.head(x)
        return y

In [5]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_epochs = 60
learning_rate = 0.01
batch_size = 256
num_classes = 10

In [6]:
class_names = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
CIFAR10_data = CIFAR10(root = 'data', train = True, transform = ToTensor(), download = True)
train_data, test_data, valid_data = torch.utils.data.random_split(CIFAR10_data, [35000, 10000, 5000])
loaders = {'train': torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle=True),
        'test': torch.utils.data.DataLoader(test_data, batch_size = batch_size, shuffle=True),
        'valid': torch.utils.data.DataLoader(valid_data, batch_size = batch_size, shuffle=True)}

100.0%


Extracting data/cifar-10-python.tar.gz to data


In [7]:
def training(model, loaders, criterion, optimizer, device):
    model.train()
    total_loss = 0.0

    for _, (images,labels) in enumerate(loaders['train']):
        images = images.requires_grad_().to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(loaders['train'])
    return avg_loss

In [15]:
def valid_or_test_fn(model, loaders, criterion, device, valid_or_test):
    model.eval()
    total_loss = 0.0
    total, correct = 0, 0
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for i, (images, labels) in enumerate(loaders[valid_or_test]):
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

            # if valid_or_test == 'test':
            #     class_names = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
            #     if i in [5, 7, 10, 12, 15, 20]:
            #         image = images[0,...].clone().squeeze().detach().cpu().numpy()
            #         label = labels[0].item()
            #         predicted_label = predicted[0].item()
            #         plt.figure(figsize=(4, 4))
            #         plt.imshow(image.transpose(1,2,0))
            #         plt.title(f'Predicted = {class_names[predicted_label]} / True Label = {class_names[label]}')
            #         plt.savefig('./test_out_1st_img_from_batch_{:04}.png'.format(i))
            #         plt.close()

        accuracy = 100 * correct / total
        avg_loss = total_loss / len(loaders[valid_or_test])
    return avg_loss, accuracy, all_labels, all_predictions

In [10]:
def plot_confusion_matrix(true_labels, predicted_labels, class_names):
    from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
    # TODO: Plot the confusion matrix, you can use the imported libraries above if desired
    y_true = np.asarray(true_labels)
    y_pred = np.asarray(predicted_labels)
    cm = confusion_matrix(y_true, y_pred, labels=list(range(len(class_names))))
    disp = ConfusionMatrixDisplay(cm, display_labels=class_names)
    
    fig, ax = plt.subplots(figsize=(6,6))
    disp.plot(ax=ax)
    plt.title("Confusion Matrix")
    plt.xticks(rotation=60)
    out_path = "ViT_confusion_matrix.png"
    plt.tight_layout()
    plt.savefig(out_path, dpi=150)
    plt.close()

In [11]:
def plot_loss(train_losses, val_losses, val_accuracies):
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Train Loss vs Epochs')
    plt.grid(True)
    plt.savefig('loss_graph.png')
    plt.close('all')

In [12]:
model = CNN_classifier(3, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

train_losses = []
val_losses = []
train_acc = []
val_accuracies = []
print('*' * 60 + '\nSTARTED Training with Batch Size = \"%s\", ' % batch_size + 'Epochs = \"%s\", ' % num_epochs + 'LR = \"%.4f\" \n' % learning_rate)
for epoch in range(num_epochs):
    # Training
    train_loss = training(model, loaders, criterion, optimizer, device)
    train_losses.append(train_loss)

    # Validation
    val_loss, valid_accuracy, _, _ = valid_or_test_fn(model, loaders, criterion, device, 'valid')
    val_losses.append(val_loss)
    val_accuracies.append(valid_accuracy)
    print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {valid_accuracy:.4f}')
plot_loss(train_losses, val_losses, val_accuracies)

************************************************************
STARTED Training with Batch Size = "256", Epochs = "60", LR = "0.0100" 

Epoch [1/60], Training Loss: 1.6696, Validation Loss: 1.7981, Validation Accuracy: 36.8400


KeyboardInterrupt: 

In [16]:
# Testing
_, test_accuracy, all_labels, all_predictions = valid_or_test_fn(model, loaders, criterion, device, 'test')
plot_confusion_matrix(all_labels, all_predictions, class_names)

print(f'\nTest Accuracy: {test_accuracy:.4f}')
print('\nFINISHED Training!\n' + '*' * 60)


Test Accuracy: 53.8900

FINISHED Training!
************************************************************
